In [ ]:
!pip install datasets
!pip install peft
# !pip install unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 32.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 29.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-n

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets, Dataset
import pandas as pd
import os
import random
import functools
import csv
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score
# from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    MistralForSequenceClassification,
    PretrainedConfig,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    AutoModelForCausalLM
)
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_location = "/content/drive/MyDrive/685 Final Project/Datasets/medmcqa-prompts-nextword"

In [ ]:
# train_dataset = load_from_disk(f"{dataset_location}/train_prompts.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts.hf")

# train_dataset = load_from_disk(f"{dataset_location}/train_prompts_micro.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_micro.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_micro.hf")

train_dataset = load_from_disk(f"{dataset_location}/train_prompts_mini.hf")
test_dataset = load_from_disk(f"{dataset_location}/test_prompts_mini.hf")
eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_mini.hf")

In [ ]:
set(train_dataset['cop'])

{0, 1, 2, 3}

In [ ]:
num_classes = len(set(train_dataset['cop']))
num_classes

4

In [ ]:
train_dataset[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': 2,
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Mul

In [ ]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'prompt_inf'],
    num_rows: 2000
})

In [ ]:
# del(model)

In [ ]:



model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
 

In [ ]:
# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj',
                      "gate_proj", "up_proj", "down_proj",],
    # target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    # task_type = 'SEQ_CLS'
)



In [ ]:
tokenizer

LlamaTokenizerFast(name_or_path='unsloth/mistral-7b-instruct-v0.2-bnb-4bit', vocab_size=32000, model_max_length=32768, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# model = SequenceClassificationWithSoftmax(model)
model

PeftModel(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit

In [ ]:
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


In [ ]:
# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['prompt'])
    return tokenized_inputs

In [ ]:
tokenized_train = train_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_train = tokenized_train.with_format('torch')
tokenized_train[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': tensor(2),
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve 

In [ ]:
# tokenized_train[0]

In [ ]:
tokenized_eval = eval_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_eval = tokenized_eval.with_format('torch')
tokenized_eval

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'prompt_inf', 'input_ids', 'attention_mask'],
    num_rows: 2000
})

In [ ]:
# tokenized_train['labels']

In [ ]:
tokenized_train['prompt'][0]

'\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: C </s>'

In [ ]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
## Adds Padding from the tokenizer.
    dict_keys = ['input_ids', 'attention_mask']
    # print(batch)
    # d = {k: [print(k, dic) for dic in batch] for k in dict_keys}
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )

    return d


In [ ]:
tokenized_train[0]

{'id': 'e9ad821a-c438-4965-9f77-760819dfa155',
 'question': 'Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma',
 'opa': 'Hyperplasia',
 'opb': 'Hyperophy',
 'opc': 'Atrophy',
 'opd': 'Dyplasia',
 'cop': tensor(2),
 'choice_type': 'single',
 'exp': 'Chronic urethral obstruction because of urinary calculi, prostatic hyperophy, tumors, normal pregnancy, tumors, uterine prolapse or functional disorders cause hydronephrosis which by definition is used to describe dilatation of renal pelvis and calculus associated with progressive atrophy of the kidney due to obstruction to the outflow of urine Refer Robbins 7yh/9,1012,9/e. P950',
 'subject_name': 'Anatomy',
 'topic_name': 'Urinary tract',
 'prompt': '\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve 

In [ ]:
# collate_fn(tokenized_train[0:8], tokenizer)
g = functools.partial(collate_fn, tokenizer=tokenizer)
g(tokenized_train.select(range(8)))

{'input_ids': tensor([[    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0],
         ...,
         [    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
out = model.generate(**g(tokenized_train.select(range(1))), max_new_tokens=5)
out

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([[    1, 28705,    13,  2287, 22478, 28747,    13,  2287,   689,  7624,
           332,   267,   362,  1650, 13364,  3112,  2940,   298,  2798,   603,
           724,  1443,  1711, 11503,   452, 28038,   541,  1736,   298,   272,
          2296,  2268,   297,  6371,  2705,   284,  4184,   338, 28724,   705,
            13,  4018, 28747,    13, 28741, 28723, 10540,   487,   452, 28038,
            13, 28760, 28723, 10540,   487,  3126, 28724,    13, 28743, 28723,
          1794,   311,  8071,    13, 28757, 28723, 25685,   452, 28038,    13,
            13,  2287,   733, 16289, 28793,   318,  5303,   456,  9713,  4191,
         27710, 22478,   304,  3084,   272,  4714,  3551,   575,   302,  2308,
          2877, 28732, 28741, 28725, 28760, 28725, 28743, 28725, 28757, 28731,
           390,   272,  4372, 28723,   733, 28748, 16289, 28793,    13,  2287,
         26307, 28747,   334, 28705,     2,   151,  1794,   311,  8071,    13]])

In [ ]:
ans = tokenizer.decode(out[0], skip_special_tokens=True)
ans

'\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: C � Atrophy\n'

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_train,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        # max_steps = 20,
        num_train_epochs = 3,# Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "answerKey",
    ),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,000 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 8
\        /    Total batch size = 256 | Total steps = 234
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.717200
2,4.758300
3,4.131500
4,4.780200
5,3.989500
6,3.148300
7,3.018700
8,2.511100
9,2.050300
10,2.173500


TrainOutput(global_step=234, training_loss=0.8599400825989552, metrics={'train_runtime': 3376.2565, 'train_samples_per_second': 17.771, 'train_steps_per_second': 0.069, 'total_flos': 5.083846003434455e+17, 'train_loss': 0.8599400825989552, 'epoch': 2.9952})

In [ ]:
# collate_fn(tokenized_train[0:8], tokenizer)
g = functools.partial(collate_fn, tokenizer=tokenizer)
g(tokenized_train.select(range(8)))

{'input_ids': tensor([[    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0],
         ...,
         [    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0],
         [    1, 28705,    13,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
out = model.generate(**g(tokenized_train.select(range(1))), max_new_tokens=5)
out

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([[    1, 28705,    13,  2287, 22478, 28747,    13,  2287,   689,  7624,
           332,   267,   362,  1650, 13364,  3112,  2940,   298,  2798,   603,
           724,  1443,  1711, 11503,   452, 28038,   541,  1736,   298,   272,
          2296,  2268,   297,  6371,  2705,   284,  4184,   338, 28724,   705,
            13,  4018, 28747,    13, 28741, 28723, 10540,   487,   452, 28038,
            13, 28760, 28723, 10540,   487,  3126, 28724,    13, 28743, 28723,
          1794,   311,  8071,    13, 28757, 28723, 25685,   452, 28038,    13,
            13,  2287,   733, 16289, 28793,   318,  5303,   456,  9713,  4191,
         27710, 22478,   304,  3084,   272,  4714,  3551,   575,   302,  2308,
          2877, 28732, 28741, 28725, 28760, 28725, 28743, 28725, 28757, 28731,
           390,   272,  4372, 28723,   733, 28748, 16289, 28793,    13,  2287,
         26307, 28747,   334, 28705,     2,   151,     2]])

In [ ]:
ans = tokenizer.decode(out[0], skip_special_tokens=True)
ans

'\n    Question:\n    Chronic urethral obstruction due to benign prismatic hyperplasia can lead to the following change in kidney parenchyma\nOptions:\nA. Hyperplasia\nB. Hyperophy\nC. Atrophy\nD. Dyplasia\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: C �'

In [ ]:
saved_model_location = "/content/drive/MyDrive/685 Final Project/Models"

In [ ]:
####### model.save_pretrained(f"{saved_model_location}/Mistral_MedMCQ_Micro_dataset")
###### model.save_pretrained(f"{saved_model_location}/Mistral_MedMCQ_Mini_dataset")
model.save_pretrained(f"{saved_model_location}/Next_Word_Mistral_MedMCQ_Mini")


In [ ]:
# model = AutoModelForSequenceClassification.from_pretrained(f"{saved_model_location}/Mistral_MedMCQ_Micro_dataset")
# tokenizer= AutoTokenizer.from_pretrained(model_name) # tokenizer of unsloth



# Evaluation


In [ ]:
dataset_location = "/content/drive/MyDrive/685 Final Project/Datasets/medmcqa-prompts-nextword"
eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts.hf")
train_dataset = load_from_disk(f"{dataset_location}/train_prompts_mini.hf")


In [ ]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'prompt_inf'],
    num_rows: 4183
})

In [ ]:
train_dataset


Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'prompt_inf'],
    num_rows: 20000
})

In [ ]:
# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['prompt_inf'])
    # tokenized_inputs['labels'] = examples['cop']
    # tokenized_inputs['labels'] = examples['label_one_hot']
    # print(f"examples['cop']: {examples['cop']}")
    return tokenized_inputs

In [ ]:
tokenized_eval = eval_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_eval = tokenized_eval.with_format('torch')
tokenized_eval

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'prompt_inf', 'input_ids', 'attention_mask'],
    num_rows: 4183
})

In [ ]:
tokenized_train = eval_dataset.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_train = tokenized_eval.with_format('torch')
tokenized_train

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'prompt_inf', 'input_ids', 'attention_mask'],
    num_rows: 4183
})

In [ ]:
collator = functools.partial(collate_fn, tokenizer=tokenizer)
# collator(tokenized_train.select(range(2)))

In [ ]:
labels_df = pd.DataFrame(tokenized_eval['cop'], columns=['true'])

In [ ]:
labels_df.head()

,true
0,0
1,0
2,2
3,2
4,0


In [ ]:
out = model.generate(**g(tokenized_eval.select(range(10))), max_new_tokens=1)
out

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([[    1, 28705,    13,  ...,     0,     0,  3312],
        [    1, 28705,    13,  ...,     0,     0, 28748],
        [    1, 28705,    13,  ..., 28705,     2,   178],
        ...,
        [    1, 28705,    13,  ...,     0,     0,    13],
        [    1, 28705,    13,  ...,     0,     0,    13],
        [    1, 28705,    13,  ...,     0,     0,    13]])

In [ ]:
ans = tokenizer.decode(out[3], skip_special_tokens=True)
ans

'\n    Question:\n    Axonal transport is:\nOptions:\nA. Antegrade\nB. Retrograde\nC. Antegrade and retrograde\nD. None\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: \n'

In [ ]:
out = model.generate(**g(tokenized_train.select(range(10))), max_new_tokens=1)
out

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


tensor([[    1, 28705,    13,  ...,     0,     0, 28748],
        [    1, 28705,    13,  ...,     0,     0, 28748],
        [    1, 28705,    13,  ..., 28705,     2,   178],
        ...,
        [    1, 28705,    13,  ...,     0,     0,    13],
        [    1, 28705,    13,  ...,     0,     0,    13],
        [    1, 28705,    13,  ...,     0,     0,    13]])

In [ ]:
ans = tokenizer.decode(out[4], skip_special_tokens=True)
ans

'\n    Question:\n    Low insulin to glucagon ratio is seen in all of these except:\nOptions:\nA. Glycogen synthesis\nB. Glycogen breakdown\nC. Gluconeogenesis\nD. Ketogenesis\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: \n'

In [ ]:
mistral_outs = []
batch_size = 32

# for i in range(0, len(tokenized_eval), batch_size):
  # print(i)
for i in range(20, 30, batch_size):

  # print(i)
  batch = tokenized_eval.select(range(i, min(i + batch_size, len(tokenized_eval))))
  collated_batch = dict(collator(batch))

  outputs = model.generate(**collated_batch, max_new_tokens=5)

  mistral_outs.extend(outputs)

  # print(batch)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [ ]:
mistral_outs[0]
ans = tokenizer.decode(mistral_outs[0], skip_special_tokens=True)
ans

'\n    Question:\n    2, 3-BPG binds to sites of haemoglobin and the affinity for oxygen\nOptions:\nA. 4, decreases\nB. 1, decreases\nC. 4, increases\nD. 1, increases\n\n    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]\n    Answer: \nOptions:\nA'

In [ ]:
ans = [tokenizer.decode(i[0], skip_special_tokens=True) for i in mistral_outs]
ans

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

# The Model Produced '\\'s instead of options.